In [1]:
#Use the same imports 
import tensorflow as tf

/trinity/shared/pkg/devel/python/3.6.5/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc 
rc('text', usetex=False) # switch tex off for using it in the ipython notebook 
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
#from plotting import newfig, savefig
from mpl_toolkits.mplot3d import Axes3D
import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [3]:
from Schrodinger import DeepHPM

### Define Domain Bounds

In [5]:
lb_idn = np.array([0.0, -5.0])
ub_idn = np.array([np.pi/2, 5.0])

lb_sol = np.array([0.0, -5.0])
ub_sol = np.array([np.pi/2, 5.0])

### Load Data

In [7]:
data_idn = scipy.io.loadmat('../Data/NLS.mat')

t_idn = data_idn['t'].flatten()[:,None]
x_idn = data_idn['x'].flatten()[:,None]
Exact_u_idn = np.real(data_idn['usol'])
Exact_v_idn = np.imag(data_idn['usol'])

T_idn, X_idn = np.meshgrid(t_idn,x_idn)

keep = 1
index = int(keep*t_idn.shape[0])
T_idn = T_idn[:,0:index]
X_idn = X_idn[:,0:index]
Exact_u_idn = Exact_u_idn[:,0:index]
Exact_v_idn = Exact_v_idn[:,0:index]
Exact_uv_idn = np.sqrt(Exact_u_idn**2 + Exact_v_idn**2)    

t_idn_star = T_idn.flatten()[:,None]
x_idn_star = X_idn.flatten()[:,None]
X_idn_star = np.hstack((t_idn_star, x_idn_star))
u_idn_star = Exact_u_idn.flatten()[:,None]
v_idn_star = Exact_v_idn.flatten()[:,None]

#

data_sol = scipy.io.loadmat('../Data/NLS.mat')

t_sol = data_sol['t'].flatten()[:,None]
x_sol = data_sol['x'].flatten()[:,None]
Exact_u_sol = np.real(data_sol['usol'])
Exact_v_sol = np.imag(data_sol['usol'])
Exact_uv_sol = np.sqrt(Exact_u_sol**2 + Exact_v_sol**2)

T_sol, X_sol = np.meshgrid(t_sol,x_sol)

t_sol_star = T_sol.flatten()[:,None]
x_sol_star = X_sol.flatten()[:,None]
X_sol_star = np.hstack((t_sol_star, x_sol_star))
u_sol_star = Exact_u_sol.flatten()[:,None]
v_sol_star = Exact_v_sol.flatten()[:,None]

###  Prepare Training Data

#### For identification

In [8]:
N_train = 10000

idx = np.random.choice(t_idn_star.shape[0], N_train, replace=False)    
t_train = t_idn_star[idx,:]
x_train = x_idn_star[idx,:]
u_train = u_idn_star[idx,:]
v_train = v_idn_star[idx,:]

noise = 0.00
u_train = u_train + noise*np.std(u_train)*np.random.randn(u_train.shape[0], u_train.shape[1])
v_train = v_train + noise*np.std(v_train)*np.random.randn(v_train.shape[0], v_train.shape[1])

#### For solution 

In [10]:
N0 = Exact_u_sol.shape[0]
N_b = Exact_u_sol.shape[1]
N_f = 20000

idx_x = np.random.choice(x_sol.shape[0], N0, replace=False)
x0_train = x_sol[idx_x,:]
u0_train = Exact_u_sol[idx_x,0:1]
v0_train = Exact_v_sol[idx_x,0:1]

idx_t = np.random.choice(t_sol.shape[0], N_b, replace=False)
tb_train = t_sol[idx_t,:]

X_f_train = lb_sol + (ub_sol-lb_sol)*lhs(2, N_f)

### Define the Model 

#### Layers

In [14]:
uv_layers = [2, 50, 50, 50, 50, 1]
pde_layers = [6, 100, 100, 1]
layers = [2, 50, 50, 50, 50, 1]

#### Create Model

In [16]:
model = DeepHPM(t_train, x_train, u_train, v_train,
                x0_train, u0_train, v0_train, tb_train, X_f_train,
                uv_layers, pde_layers,
                layers,
                lb_idn, ub_idn,
                lb_sol, ub_sol)
    

### Training

#### Training the identifiers

In [ ]:
model.idn_uv_train(N_iter=0)
model.idn_fg_train(N_iter=0)



Loss: 8.197545e+03
Loss: 3.419573e+04
Loss: 6.572091e+03
Loss: 6.547103e+03
Loss: 6.529850e+03
Loss: 6.529506e+03
Loss: 6.527536e+03
Loss: 6.527292e+03
Loss: 6.526921e+03
Loss: 6.522803e+03
Loss: 6.515484e+03
Loss: 6.498414e+03
Loss: 6.460387e+03
Loss: 1.302590e+05
Loss: 6.453696e+03
Loss: 6.423188e+03
Loss: 6.349210e+03
Loss: 6.596746e+03
Loss: 6.138188e+03
Loss: 6.193115e+03
Loss: 6.025228e+03
Loss: 5.858675e+03
Loss: 6.611517e+03
Loss: 5.735683e+03
Loss: 5.713444e+03
Loss: 5.690479e+03
Loss: 5.706717e+03
Loss: 5.660221e+03
Loss: 5.629188e+03
Loss: 5.620299e+03
Loss: 5.609371e+03
Loss: 5.590385e+03
Loss: 5.548232e+03
Loss: 5.518576e+03
Loss: 5.500272e+03
Loss: 5.492656e+03
Loss: 5.476072e+03
Loss: 5.365790e+03
Loss: 5.277876e+03
Loss: 5.178044e+03
Loss: 5.080987e+03
Loss: 5.036181e+03
Loss: 4.954189e+03
Loss: 4.923822e+03
Loss: 4.871221e+03
Loss: 4.825811e+03
Loss: 4.796172e+03
Loss: 4.744128e+03
Loss: 4.623774e+03
Loss: 4.505899e+03
Loss: 4.410060e+03
Loss: 4.268415e+03
Loss: 4.1445

Loss: 8.115572e-01
Loss: 8.090317e-01
Loss: 8.053662e-01
Loss: 8.059320e-01
Loss: 8.034390e-01
Loss: 8.016323e-01
Loss: 7.999719e-01
Loss: 7.980065e-01
Loss: 7.933003e-01
Loss: 8.118422e-01
Loss: 7.918221e-01
Loss: 7.879378e-01
Loss: 7.843940e-01
Loss: 7.817571e-01
Loss: 7.784373e-01
Loss: 7.744937e-01
Loss: 7.703854e-01
Loss: 7.651318e-01
Loss: 7.599612e-01
Loss: 7.529836e-01
Loss: 7.462854e-01
Loss: 7.752200e-01
Loss: 7.443444e-01
Loss: 7.415600e-01
Loss: 7.394950e-01
Loss: 7.362438e-01
Loss: 7.354579e-01
Loss: 7.315807e-01
Loss: 7.293153e-01
Loss: 7.269340e-01
Loss: 7.238671e-01
Loss: 7.181175e-01
Loss: 7.235850e-01
Loss: 7.161838e-01
Loss: 7.129456e-01
Loss: 7.104071e-01
Loss: 7.069919e-01
Loss: 7.037150e-01
Loss: 6.999942e-01
Loss: 6.955495e-01
Loss: 6.927896e-01
Loss: 6.886963e-01
Loss: 6.853886e-01
Loss: 6.837613e-01
Loss: 6.814772e-01
Loss: 6.801853e-01
Loss: 6.781615e-01
Loss: 6.758513e-01
Loss: 6.723726e-01
Loss: 6.700158e-01
Loss: 6.662747e-01
Loss: 6.636877e-01
Loss: 6.6153

Loss: 1.906674e-01
Loss: 1.904601e-01
Loss: 1.902002e-01
Loss: 1.897674e-01
Loss: 1.895060e-01
Loss: 1.891586e-01
Loss: 1.888977e-01
Loss: 1.885773e-01
Loss: 1.879347e-01
Loss: 1.878384e-01
Loss: 1.870786e-01
Loss: 1.868206e-01
Loss: 1.863681e-01
Loss: 1.860849e-01
Loss: 1.856258e-01
Loss: 1.851826e-01
Loss: 1.848359e-01
Loss: 1.847363e-01
Loss: 1.842017e-01
Loss: 1.839582e-01
Loss: 1.836076e-01
Loss: 1.832310e-01
Loss: 1.834825e-01
Loss: 1.830255e-01
Loss: 1.825548e-01
Loss: 1.822268e-01
Loss: 1.818403e-01
Loss: 1.815382e-01
Loss: 1.820676e-01
Loss: 1.813351e-01
Loss: 1.810279e-01
Loss: 1.806860e-01
Loss: 1.803251e-01
Loss: 1.797033e-01
Loss: 1.787417e-01
Loss: 1.778384e-01
Loss: 1.772113e-01
Loss: 1.766979e-01
Loss: 1.761855e-01
Loss: 1.756025e-01
Loss: 1.751440e-01
Loss: 1.746744e-01
Loss: 1.742319e-01
Loss: 1.738890e-01
Loss: 1.736133e-01
Loss: 1.730849e-01
Loss: 1.731766e-01
Loss: 1.727704e-01
Loss: 1.721644e-01
Loss: 1.716935e-01
Loss: 1.713798e-01
Loss: 1.710440e-01
Loss: 1.7075

Loss: 8.365580e-02
Loss: 8.356878e-02
Loss: 8.328922e-02
Loss: 8.316606e-02
Loss: 8.295458e-02
Loss: 8.283045e-02
Loss: 8.256436e-02
Loss: 8.244304e-02
Loss: 8.229052e-02
Loss: 8.205678e-02
Loss: 8.207528e-02
Loss: 8.191970e-02
Loss: 8.173467e-02
Loss: 8.163109e-02
Loss: 8.151841e-02
Loss: 8.139594e-02
Loss: 8.130876e-02
Loss: 8.120491e-02
Loss: 8.106149e-02
Loss: 8.080706e-02
Loss: 8.126894e-02
Loss: 8.074568e-02
Loss: 8.061180e-02
Loss: 8.046178e-02
Loss: 8.024818e-02
Loss: 8.000102e-02
Loss: 7.985345e-02
Loss: 7.975551e-02
Loss: 7.969715e-02
Loss: 7.964199e-02
Loss: 7.955019e-02
Loss: 7.946690e-02
Loss: 7.933872e-02
Loss: 7.925999e-02
Loss: 7.917144e-02
Loss: 7.902142e-02
Loss: 7.883976e-02
Loss: 7.867844e-02
Loss: 7.858516e-02
Loss: 7.848786e-02
Loss: 7.840163e-02
Loss: 7.827862e-02
Loss: 7.809781e-02
Loss: 7.799082e-02
Loss: 7.788302e-02
Loss: 7.776938e-02
Loss: 7.829128e-02
Loss: 7.771370e-02
Loss: 7.756419e-02
Loss: 7.747149e-02
Loss: 7.732059e-02
Loss: 7.749206e-02
Loss: 7.7270

Loss: 4.873353e-02
Loss: 4.871130e-02
Loss: 4.866755e-02
Loss: 4.862807e-02
Loss: 4.858860e-02
Loss: 4.854463e-02
Loss: 4.851679e-02
Loss: 4.844669e-02
Loss: 4.838832e-02
Loss: 4.835396e-02
Loss: 4.829970e-02
Loss: 4.827301e-02
Loss: 4.821612e-02
Loss: 4.816159e-02
Loss: 4.810591e-02
Loss: 4.803491e-02
Loss: 4.805268e-02
Loss: 4.801050e-02
Loss: 4.797450e-02
Loss: 4.790173e-02
Loss: 4.784761e-02
Loss: 4.780070e-02
Loss: 4.776564e-02
Loss: 4.772164e-02
Loss: 4.766466e-02
Loss: 4.760960e-02
Loss: 4.749582e-02
Loss: 4.741094e-02
Loss: 4.727963e-02
Loss: 4.722052e-02
Loss: 4.716358e-02
Loss: 4.712557e-02
Loss: 4.709694e-02
Loss: 4.705479e-02
Loss: 4.698793e-02
Loss: 4.692530e-02
Loss: 4.682203e-02
Loss: 4.688955e-02
Loss: 4.678483e-02
Loss: 4.672597e-02
Loss: 4.668558e-02
Loss: 4.662789e-02
Loss: 4.658601e-02
Loss: 4.651749e-02
Loss: 4.646639e-02
Loss: 4.642241e-02
Loss: 4.636442e-02
Loss: 4.630696e-02
Loss: 4.627144e-02
Loss: 4.622770e-02
Loss: 4.618521e-02
Loss: 4.614118e-02
Loss: 4.6088

...

Loss: 3.324428e-01
Loss: 3.324398e-01
Loss: 3.324343e-01
Loss: 3.324288e-01
Loss: 3.324206e-01
Loss: 3.324143e-01
Loss: 3.324044e-01
Loss: 3.324496e-01
Loss: 3.324029e-01
Loss: 3.323962e-01
Loss: 3.323881e-01
Loss: 3.323801e-01
Loss: 3.323742e-01
Loss: 3.323688e-01
Loss: 3.323641e-01
Loss: 3.323617e-01
Loss: 3.323522e-01
Loss: 3.323655e-01
Loss: 3.323475e-01
Loss: 3.323398e-01
Loss: 3.323340e-01
Loss: 3.323280e-01
Loss: 3.323246e-01
Loss: 3.323185e-01
Loss: 3.323129e-01
Loss: 3.323067e-01
Loss: 3.322958e-01
Loss: 3.322975e-01
Loss: 3.322890e-01
Loss: 3.322764e-01
Loss: 3.322703e-01
Loss: 3.322640e-01
Loss: 3.322577e-01
Loss: 3.322582e-01
Loss: 3.322533e-01
Loss: 3.322480e-01
Loss: 3.322439e-01
Loss: 3.322375e-01
Loss: 3.322298e-01
Loss: 3.322209e-01
Loss: 3.322141e-01
Loss: 3.322060e-01
Loss: 3.321994e-01
Loss: 3.321939e-01
Loss: 3.321843e-01
Loss: 3.321801e-01
Loss: 3.321758e-01
Loss: 3.321714e-01
Loss: 3.321657e-01
Loss: 3.321580e-01
Loss: 3.321545e-01
Loss: 3.321502e-01
Loss: 3.3214

Loss: 3.302857e-01
Loss: 3.302795e-01
Loss: 3.302766e-01
Loss: 3.302704e-01
Loss: 3.302866e-01
Loss: 3.302682e-01
Loss: 3.302643e-01
Loss: 3.302585e-01
Loss: 3.302507e-01
Loss: 3.302491e-01
Loss: 3.302410e-01
Loss: 3.302366e-01
Loss: 3.302322e-01
Loss: 3.302275e-01
Loss: 3.302218e-01
Loss: 3.302121e-01
Loss: 3.302090e-01
Loss: 3.302041e-01
Loss: 3.302010e-01
Loss: 3.301967e-01
Loss: 3.301924e-01
Loss: 3.301894e-01
Loss: 3.301810e-01
Loss: 3.301814e-01
Loss: 3.301768e-01
Loss: 3.301705e-01
Loss: 3.301665e-01
Loss: 3.301615e-01
Loss: 3.301582e-01
Loss: 3.301544e-01
Loss: 3.301491e-01
Loss: 3.301424e-01
Loss: 3.301376e-01
Loss: 3.301336e-01
Loss: 3.301312e-01
Loss: 3.301270e-01
Loss: 3.301180e-01
Loss: 3.301121e-01
Loss: 3.301038e-01
Loss: 3.300998e-01
Loss: 3.300951e-01
Loss: 3.300903e-01
Loss: 3.300832e-01
Loss: 3.300960e-01
Loss: 3.300795e-01
Loss: 3.300732e-01
Loss: 3.300691e-01
Loss: 3.300633e-01
Loss: 3.300561e-01
Loss: 3.300469e-01
Loss: 3.300418e-01
Loss: 3.300350e-01
Loss: 3.3003

Loss: 3.280940e-01
Loss: 3.280838e-01
Loss: 3.280989e-01
Loss: 3.280800e-01
Loss: 3.280728e-01
Loss: 3.280690e-01
Loss: 3.280594e-01
Loss: 3.280591e-01
Loss: 3.280557e-01
Loss: 3.280492e-01
Loss: 3.280445e-01
Loss: 3.280371e-01
Loss: 3.280312e-01
Loss: 3.280275e-01
Loss: 3.280239e-01
Loss: 3.280210e-01
Loss: 3.280190e-01
Loss: 3.280131e-01
Loss: 3.280174e-01
Loss: 3.280095e-01
Loss: 3.280041e-01
Loss: 3.279981e-01
Loss: 3.279952e-01
Loss: 3.279919e-01
Loss: 3.279886e-01
Loss: 3.279824e-01
Loss: 3.279774e-01
Loss: 3.279684e-01
Loss: 3.279635e-01
Loss: 3.279602e-01
Loss: 3.279577e-01
Loss: 3.279542e-01
Loss: 3.279502e-01
Loss: 3.279465e-01
Loss: 3.279431e-01
Loss: 3.279407e-01
Loss: 3.279341e-01
Loss: 3.279287e-01
Loss: 3.279238e-01
Loss: 3.279203e-01
Loss: 3.279167e-01
Loss: 3.279086e-01
Loss: 3.279038e-01
Loss: 3.279006e-01
Loss: 3.278992e-01
Loss: 3.278964e-01
Loss: 3.278907e-01
Loss: 3.278878e-01
Loss: 3.278806e-01
Loss: 3.278780e-01
Loss: 3.278732e-01
Loss: 3.278680e-01
Loss: 3.2786

Loss: 3.260845e-01
Loss: 3.260809e-01
Loss: 3.260731e-01
Loss: 3.260818e-01
Loss: 3.260692e-01
Loss: 3.260617e-01
Loss: 3.260571e-01
Loss: 3.260525e-01
Loss: 3.260489e-01
Loss: 3.260453e-01
Loss: 3.260419e-01
Loss: 3.260386e-01
Loss: 3.260311e-01
Loss: 3.260226e-01
Loss: 3.260170e-01
Loss: 3.260113e-01
Loss: 3.260091e-01
Loss: 3.260062e-01
Loss: 3.260026e-01
Loss: 3.259968e-01
Loss: 3.259941e-01
Loss: 3.259924e-01
Loss: 3.259868e-01
Loss: 3.259842e-01
Loss: 3.259810e-01
Loss: 3.259765e-01
Loss: 3.259739e-01
Loss: 3.259709e-01
Loss: 3.259659e-01
Loss: 3.259614e-01
Loss: 3.259550e-01
Loss: 3.259531e-01
Loss: 3.259498e-01
Loss: 3.259437e-01
Loss: 3.259407e-01
Loss: 3.259329e-01
Loss: 3.259289e-01
Loss: 3.259251e-01
Loss: 3.259222e-01
Loss: 3.259187e-01
Loss: 3.259119e-01
Loss: 3.259267e-01
Loss: 3.259099e-01
Loss: 3.259066e-01
Loss: 3.259015e-01
Loss: 3.258948e-01
Loss: 3.258878e-01
Loss: 3.258821e-01
Loss: 3.258754e-01
Loss: 3.258708e-01
Loss: 3.258676e-01
Loss: 3.258619e-01
Loss: 3.2585

Loss: 3.242338e-01
Loss: 3.242312e-01
Loss: 3.242270e-01
Loss: 3.242243e-01
Loss: 3.242165e-01
Loss: 3.242226e-01
Loss: 3.242124e-01
Loss: 3.242090e-01
Loss: 3.242061e-01
Loss: 3.242028e-01
Loss: 3.241971e-01
Loss: 3.241959e-01
Loss: 3.241899e-01
Loss: 3.241877e-01
Loss: 3.241843e-01
Loss: 3.241797e-01
Loss: 3.241739e-01
Loss: 3.241693e-01
Loss: 3.241653e-01
Loss: 3.241611e-01
Loss: 3.241563e-01
Loss: 3.241508e-01
Loss: 3.241459e-01
Loss: 3.241414e-01
Loss: 3.241344e-01
Loss: 3.241335e-01
Loss: 3.241278e-01
Loss: 3.241263e-01
Loss: 3.241231e-01
Loss: 3.241213e-01
Loss: 3.241183e-01
Loss: 3.241155e-01
Loss: 3.241134e-01
Loss: 3.241090e-01
Loss: 3.241057e-01


This two lines trains the two identifier networks. The number of iterations is set to zero means there is no adam iteration

#### Training the solver

In [ ]:
model.sol_train(N_iter=0)